# TP6

### `Resolver usando Pandas`

Resolver los ejercicios del TP3 utilizando la librería Pandas.

### Ejercicio 1: Cargar Datos de ventas.

El archivo datos.dat contiene el registro de las ventas realizadas.

Tiene un formato de ancho fijo:
- `fecha`:    10 lugares
- `producto`: 30 lugares
- `precio`:   10 lugares
- `cantidad`:  5 lugares

Hacer una funcion que cargue los datos en un DataFrame de Pandas.

In [ ]:
import pandas as pd

def cargar_datos():
    colspecs = [(0, 10), (10, 40), (40, 50), (50, 55)]
    
    column_names = ['fecha', 'producto', 'precio', 'cantidad']
    
    df = pd.read_fwf('datos.dat', colspecs=colspecs, names=column_names)
    
    df['precio'] = pd.to_numeric(df['precio'], errors='coerce')
    df['cantidad'] = pd.to_numeric(df['cantidad'], errors='coerce')
    
    return df

datos = cargar_datos()
print(datos)

### Ejercicio 2: Calcular el total de ventas.
Hacer una función que sume los importes vendidos (precio * cantidad) y las cantidades.


In [ ]:
def calcular_totales(datos): 
    importe = sum(datos["precio"] * datos["cantidad"])
    
    cantidad = sum(datos["cantidad"])
    
    return (importe,cantidad)

importe, cantidad = calcular_totales(datos)

print(f"Las ventas fueron de ${importe:2f} en {cantidad} unidades")

### Ejercicio 3: Listar las unidades vendidas.
Listar cuántas unidades se vendieron en total para cada producto


In [ ]:
def unidades_vendidas(datos):
    return datos.groupby("producto").agg({"cantidad" : "sum"})
    
unidades_vendidas(datos)

###  Ejercicio 4: Listar el precio promedio por producto.
Hacer un listado del precio promedio por producto.


In [ ]:
def precio_promedio(datos):
    return datos.groupby("producto").agg({"precio" : "mean"})
    
precio_promedio(datos)

### Ejercicio 5: Ranking de productos
Realizar un listado de los 3 productos más vendidos ordenados por la cantidad de unidades vendidas (ordenadas de mayor a menor)


In [ ]:
def ranking_productos(datos, top=3):
    df_productos_cantidades = datos.groupby("producto").agg({"cantidad" : "sum"})
    df_ranking = df_productos_cantidades.sort_values(by = 'cantidad', ascending = False)
    return df_ranking[:top:]

ranking_productos(datos)

### Ejercicio 6: Listar las ventas por mes
Realizar un listado del total de unidades vendidas por producto separado por mes.


In [ ]:
def ventas_por_mes(datos):
    datos['fecha'] = pd.to_datetime(datos['fecha'], errors='coerce')
    
    datos['mes'] = datos['fecha'].dt.to_period('M')  
    
    ventas = datos.groupby(['mes', 'producto'])['cantidad'].sum().unstack(fill_value=0)
    
    return ventas
     

ventas_por_mes(datos)

### Ejercicio 7: Informe general

Mostrar un listado de productos ordenados alfabeticamente que contengan el precio promedio, la cantidad de unidades vendidas y el importe total vendido para cada producto

In [ ]:
def resumen_ventas(datos): 
    datos['importe'] = datos['precio'] * datos['cantidad']
    
    resumen = datos.groupby('producto').agg(
        precio_promedio=('precio', 'mean'),
        unidades_vendidas=('cantidad', 'sum'),
        importe_total=('importe', 'sum')
    ).sort_index()
    return resumen

print(resumen_ventas(datos))

## `Resolver usando NumPy`
## Resolver el ejercicio 2 del tp1 usando NumPy

### Ejercicio 8

Escribe una función en Python que encuentre los valores de `a`, `b`, y `c` para que la función cuadrática `f(x) = a x^2 + b x + c` pase exactamente por los siguientes puntos:

| x  | y  |
|---:|---:|
|  0 |  0 |
|  1 |  8 |
|  2 | 12 |
|  3 | 12 |
|  5 |  0 |

### Requisitos:
- La función debe explorar posibles valores de `a`, `b`, y `c` utilizando un método de prueba y error.
- Debe devolver los valores que hagan que la diferencia entre la función `f(x)` y los valores medidos `y` sea exactamente cero para cada punto.

> **Pista**: Los valores de `a`, `b`, y `c` son números pequeños.

La idea es implementar el mismo algoritmo que se uso en el TP1 pero usando NumPy en lugar de Python puro.

In [ ]:
import numpy as np

def f(x, coeficientes):
    a, b, c = coeficientes
    return a * x**2 + b * x + c
    
def error(y, y_pred):
    return np.sum(np.abs(y - y_pred))
    
X = np.array([0, 1, 2, 3, 5])
Y = np.array([0, 8, 12, 12, 0])

def buscar_coeficientes():
    mejor_error = float('inf')
    mejores_coeficientes = None
    
    for a in np.linspace(-5, 5, 100):
        for b in np.linspace(-10, 10, 100):
            for c in np.linspace(-10, 10, 100):
                Y_pred = f(X, (a, b, c))
                e = error(Y, Y_pred)
                if e < mejor_error:
                    mejor_error = e
                    mejores_coeficientes = (a, b, c)
                    
    return mejores_coeficientes

coeficientes = buscar_coeficientes()
coeficientes

### Ejercicio 9: Resolver el ejercicio 3 del TP1 usando NumPy
Buscar los coeficientes de la función que minimice la suma de los cuadrados de las diferencias entre los valores medidos y los valores de la función.

1. Crear un array con los coeficientes elegidos al azar (usar `randint(-10,10,3)`).
2. Calcular el valor de la función y el error correspondiente.
3. Mientras que el error sea mayor a 1:
    1. Definir nuevos coeficientes agregándoles un pequeño valor al azar a los coeficientes actuales (aprendizaje = 0.001).
    2. Si el error para los nuevos coeficientes es menor que el anterior, reemplazar los coeficientes actuales por los nuevos.


In [ ]:
import numpy as np
from numpy.random import randint

def f(x, coeficientes):
    a, b, c = coeficientes
    return a * x**2 + b * x + c

def error(y, y_pred):
    return np.sum((y - y_pred)**2)

X = np.array([0, 1, 2, 3, 5])
Y = np.array([0, 8, 12, 11, 1])

def buscar_coeficientes():
    coeficientes = randint(-10, 10, 3)
    mejor_error = error(Y, f(X, coeficientes))
    aprendizaje = 0.001

    while mejor_error > 1:
        nuevos_coeficientes = coeficientes + aprendizaje * randint(-10, 10, 3)
        nuevo_error = error(Y, f(X, nuevos_coeficientes))
        
        if nuevo_error < mejor_error:
            coeficientes = nuevos_coeficientes
            mejor_error = nuevo_error

    return coeficientes


coeficientes = buscar_coeficientes()
coeficientes